In [4]:
import vertex_simulation.primitives as primitives
import vertex_simulation.simulation as simulation
import matplotlib.pyplot as plt
import numpy as np
import torch

In [5]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
dtype = torch.float32
print(f'device: {device}\ndtype: {dtype}')

device: cpu
dtype: torch.float32


In [8]:
np.random.seed(42)# np RNG seed
torch.manual_seed(0);# torch RNG seed
v_x,regions =primitives.unit_hexagons(4,4) # unit hexagons
v_x += np.random.randn(v_x.shape[0], v_x.shape[1])*.2 #perturb vertices
edge_list,cells = primitives.VoronoiRegions2Edges(regions) #convert Voronoi regions to cells & edges
# Define cell monolayer
cell_graph = primitives.Monolayer(vertices=primitives.Vertex(v_x.copy().tolist(),dtype=dtype), 
                       edges=torch.tensor(edge_list), 
                       cells=cells)
cell_graph.to_(device) # to plot use --> plot_graph_as_quiver(cell_graph)

In [13]:
omega_0 = torch.tensor([np.pi/2],dtype=dtype,device=device)
phase = torch.rand((len(edge_list),1)).type(dtype).to(device)
p_keep = 0.75 # fraction to keep active
e_ij_on = (torch.rand((len(edge_list),1))<p_keep).type(dtype).to(device)
omega = torch.randn_like(e_ij_on) + omega_0

sim = simulation.Simulation_Honda(m = cell_graph, params = {'Ka':1., 'A0':2.3,'Kp':0.003,'P0':0.,
                                                 'Lambda_ij': lambda m,t: e_ij_on*torch.cos(omega*t+phase)**2 })
t, verts_t, Energies_maxSpeeds = sim.sample_trajectory(T=1000, delta_T=0.001, sample_freq=10, print_freq=200)

iter:200; t=0.199: E=23.82; max_spd = 1.3
iter:400; t=0.399: E= 13.8; max_spd =   1
iter:600; t=0.599: E=11.61; max_spd = 1.1
iter:800; t=0.799: E=14.19; max_spd = 1.4
iter:1000; t=0.999: E=18.38; max_spd = 0.95
